In [ ]:
import matplotlib.pyplot as plt
from Genetic.Representation import Function
from Genetic.NeuroEvolution import NeuroEvolution
import tensorflow.keras as keras
from tensorflow.keras import layers
from tensorflow.keras.losses import mse
import tensorflow as tf
import tensorflow.keras.backend as K
import pandas as pd
import seaborn as sns
from IPython.display import display
import numpy as np
from Model.model import SurvModel, FlchainModel, NwtcoModel, SupportModel
from Data.dataset import Flchain, Support, Nwtco
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from scipy.stats import f_oneway, ttest_rel

from Utils.utils import Experiment, FunctionHelper, StatsHelper
from itertools import combinations

In [ ]:
surv_reg1_func = [1, 9, 9, 1, 5, 12, 7, 9, 7]
surv_reg2_func = [1, 9, 12, 1, 5, 12, 7, 9, 7]

surv_reg1_f = FunctionHelper(surv_reg1_func)
surv_reg2_f = FunctionHelper(surv_reg2_func)

surv_reg1_f.plot_surv_loss()
surv_reg2_f.plot_surv_loss()

In [ ]:
model_class = NwtcoModel
ds = Nwtco('Data/nwtco.csv')

In [ ]:
(x_train, ye_train, y_train, e_train,
 x_val, ye_val, y_val, e_val,
 x_test, ye_test, y_test, e_test) = ds.get_train_val_test_from_splits(test_id=0, val_id=1)
xy = (x_train, ye_train), (x_val, ye_val), (x_test, ye_test)

In [ ]:
exp_fix_left = [    
    [
    [6, 8, 24, 1, 6, 23, 27, 27, 13],
    [0.7159451, 0.7147979, 0.70356005, 0.654633, 0.6085865]
    ]
    ,
    [
    [2, 13, 17, 3, 3, 16, 24, 27, 26],
    [0.7398461, 0.7456392, 0.7024854, 0.6869316, 0.6461919]
    ]
    ,
    [
    [2, 29, 17, 4, 2, 22, 23, 25, 28],
    [0.7532277, 0.74121404, 0.71607333, 0.69473165, 0.65950316]
    ]
    ,
    [
    [2, 15, 24, 1, 4, 27, 7, 17, 28],
    [0.75763243, 0.75088567, 0.72227436, 0.6933645, 0.6526281]
    ]
    ,
    [
    [1, 14, 12, 4, 6, 9, 17, 26, 15],
    [0.75110894, 0.7523076, 0.7198409, 0.69285184, 0.6572115]
    ]
    ,
    [
    [5, 9, 19, 2, 3, 28, 12, 17, 26],
    [0.75421894, 0.73794115, 0.72266966, 0.688372, 0.6419499]
    ]
    ,
    [
    [2, 8, 17, 2, 2, 27, 22, 25, 23],
    [0.7558049, 0.7504811, 0.7117684, 0.6929495, 0.6526281]
    ]
    ,
    [
    [5, 17, 26, 4, 2, 15, 23, 9, 7],
    [0.7562076, 0.75126565, 0.712769, 0.6977955, 0.64961725]
    ]
    ,
    [
    [4, 24, 26, 2, 5, 28, 28, 14, 21],
    [0.73821056, 0.7467547, 0.6965128, 0.68730396, 0.6504461]
    ]
    ,
    [
    [3, 26, 14, 2, 1, 21, 30, 15, 28],
    [0.75919986, 0.7484708, 0.71755564, 0.6922049, 0.6469599]
    ]
]


exp_fix_right = [
    [
    [5, 12, 19, 4, 2, 8, 26, 26, 11],
    [0.7361538, 0.7425992, 0.7248561, 0.68159735, 0.62622505]
    ]
    ,
    [
    [6, 13, 28, 1, 2, 10, 23, 7, 13],
    [0.729463, 0.7391547, 0.72110087, 0.67584807, 0.6316861]
    ]
    ,
    [
    [1, 15, 27, 2, 1, 23, 19, 25, 30],
    [0.73288274, 0.7435921, 0.70992166, 0.6748227, 0.6335755]
    ]
    ,
    [
    [6, 15, 19, 1, 4, 10, 10, 21, 13],
    [0.7278275, 0.7419373, 0.7047212, 0.68137765, 0.6285777]
    ]
    ,
    [
    [1, 16, 21, 1, 6, 17, 18, 13, 22],
    [0.70629305, 0.7112431, 0.69296145, 0.655048, 0.6129504]
    ]
    ,
    [
    [3, 15, 11, 4, 5, 8, 15, 12, 10],
    [0.73204017, 0.74229276, 0.711824, 0.68471, 0.62851673]
    ]
    ,
    [
    [2, 12, 17, 2, 6, 23, 30, 13, 8],
    [0.7154, 0.73790437, 0.7129234, 0.67244244, 0.628468]
    ]
    ,
    [
    [4, 15, 27, 2, 1, 23, 27, 23, 8],
    [0.72980994, 0.74311405, 0.7128369, 0.67825276, 0.62922376]
    ]
    ,
    [
    [3, 12, 8, 2, 6, 23, 27, 17, 10],
    [0.7260928, 0.74266046, 0.7048941, 0.6721861, 0.6283461]
    ]
    ,
    [
    [5, 10, 10, 1, 5, 25, 20, 30, 22],
    [0.7260433, 0.7368869, 0.70520294, 0.6612124, 0.6207397]
    ]
]

exp_opt_both = [
    [
    [2, 24, 26, 3, 5, 23, 8, 26, 20],
    [0.75557566, 0.74896115, 0.6931529, 0.68972695, 0.6355624]
    ]
    ,
    [
    [3, 24, 9, 2, 3, 10, 25, 23, 26],
    [0.7465369, 0.75015014, 0.708952, 0.6798945, 0.62873614]
    ]
    ,
    [
    [4, 13, 17, 5, 3, 11, 23, 23, 20],
    [0.74186575, 0.74427855, 0.7164933, 0.6981006, 0.6556634]
    ]
    ,
    [
    [3, 12, 9, 3, 3, 27, 23, 26, 19],
    [0.74873, 0.73890954, 0.7080811, 0.68436825, 0.6495441]
    ]
    ,
    [
    [6, 10, 11, 2, 4, 28, 11, 15, 17],
    [0.7571554, 0.74746567, 0.72580105, 0.68065745, 0.6355624]
    ]
    ,
    [
    [3, 11, 19, 1, 3, 20, 29, 28, 30],
    [0.71743816, 0.7270805, 0.6929923, 0.6705809, 0.6268102]
    ]
    ,
    [
    [1, 10, 15, 1, 6, 25, 9, 22, 21],
    [0.74350125, 0.7491695, 0.701911, 0.6761471, 0.6336547]
    ]
    ,
    [
    [1, 19, 12, 6, 5, 12, 12, 13, 26],
    [0.7536118, 0.7524179, 0.70849496, 0.6903617, 0.654676]
    ]
    ,
    [
    [5, 11, 30, 1, 4, 26, 25, 19, 23],
    [0.7558606, 0.7575663, 0.7106876, 0.6742978, 0.6363791]
    ]
    ,
    [
    [6, 25, 17, 2, 1, 12, 24, 24, 7],
    [0.7357821, 0.7412753, 0.723522, 0.6844415, 0.6525184]
    ]
]

# 5 Folds CV

In [ ]:
for i in range(10):
    for j in range(10):
        func_l, folds5_l = exp_fix_left[i]
        func_r, folds5_r = exp_fix_right[j]
        StatsHelper.plot_box(folds1=folds5_l, name1=f'fix_left_{i}', folds2=folds5_r, name2=f'fix_right_{j}')
    print('--------------------------------------')
        